In [1]:
using KnetOnnx

┌ Info: Recompiling stale cache file /Users/egeersu/.julia/compiled/v1.2/KnetOnnx/gz8hJ.ji for KnetOnnx [5598a57d-f0b8-44c7-a305-14fa4de76fa0]
└ @ Base loading.jl:1240
  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **



In [2]:
graph = ONNXtoGraph("model_gather.onnx")
PrintGraph(graph)

model inputs: ["0"]
model outputs: ["4"]
(op1) Constant
	output1: 1
(op2) Gather
	input1: 0
	input2: 1
	output1: 2
(op3) Constant
	output1: 3
(op4) Gather
	input1: 2
	input2: 3
	output1: 4


In [3]:
model = KnetModel(graph);

In [4]:
x1 = randn(2,3)

2×3 Array{Float64,2}:
 -2.05775  -1.29032  -0.389631
 -2.66728   1.14566   0.662499

In [5]:
model(x1)

2×3 Array{Float64,2}:
 -2.05775  -1.29032  -0.389631
 -2.66728   1.14566   0.662499

In [6]:
model.tensors

Dict{Any,Any} with 5 entries:
  "4" => [-2.05775 -1.29032 -0.389631; -2.66728 1.14566 0.662499]
  "1" => Float32[1.4013e-45, 0.0]
  "0" => [-2.05775 -1.29032 -0.389631; -2.66728 1.14566 0.662499]
  "2" => [-2.05775 -1.29032 -0.389631; -2.66728 1.14566 0.662499]
  "3" => Float32[2.8026e-45, 0.0]

In [ ]:
g1 = graph.node[2]
g2 = converter_gather(g1, graph)[2]

In [ ]:
g2([1.19084 -0.0401237 0.0804831; 1.67733 -2.35425 -1.873], Float32[0.0, 0.0])

In [ ]:
struct Gather
    axis #increment before adding (julia 0->1)
end

function (g::Gather)(data, indices)
    indices_size = size(indices)

    indices = (x->(x+1)).(indices) # increment for Julia
    indices = (x->(Int32(x))).(indices) # set floats to Int for bug-free indexing

    if length(indices_size) == 1
        return gather_rank1(data, indices, g)
    end
    if length(indices_size) == 2
        return gather_rank2(data, indices, g)
    end
    if length(indices_size) > 2
        print("Gather for indices with rank > 2 are not implemented yet.")
    end
end


function gather_rank1(data, indices, layer)
    new_data = []
    axis1 = size(indices)[1]
    for a1 in (1:axis1)
        current_index = indices[a1]
        #get_data = data[:,current_index]
        get_data = data[layer.axis, current_index]
        push!(new_data, get_data)
    end
    new_data
end

function gather_rank2(data, indices, layer)
    new_data = []
    axis1, axis2 = size(indices)

    for a1 in (1:axis1)
        mini_list = []
        for a2 in (1:axis2)
            current_index = indices[a1,a2]
            get_data = data[:,current_index]
            push!(mini_list, get_data)
        end
        push!(new_data, mini_list)
    end
    new_data
end

function converter_gather(node, g)
    args = node.input
    outs = node.output
    axis = node.attribute[:axis] +1 #+1 is for Julia
    #layer = KL.Gather(axis)
    layer = Gather(axis)
    (args, layer, outs)
end